#  RandomForest

In [ ]:
import pandas as pd   # ver.1 заниженные прогнозы
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime, timedelta

# Загрузка данных о продажах из Excel
df = pd.read_excel("sales.xlsx")
df['Дата'] = pd.to_datetime(df['Дата'], format="%d.%m.%Y")

# Загрузка данных о ценах из Excel
prices_df = pd.read_excel("new_prices.xlsx")

# Получение последней даты в исходных данных
last_date = df['Дата'].max()

# Создание датафрейма с данными о ценах на все 14 дней
forecast_dates = [last_date + timedelta(days=i) for i in range(1, 15)]
forecast_data = pd.DataFrame({'Дата': forecast_dates})

# Дублируем данные о ценах для каждой номенклатуры
forecast_data['Цена'] = prices_df['Цена'].iloc[0]

# Разделение данных на признаки и целевую переменную
X = df[['Цена']]
y = df['Кол-во продано, шт.']

# Обучение модели случайного леса для каждой номенклатуры
models = {}
for item in df['Номенклатура'].unique():
    item_df = df[df['Номенклатура'] == item]
    X_item = item_df[['Цена']]
    y_item = item_df['Кол-во продано, шт.']
    model = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_item, y_item)
    models[item] = model

# Создание списка для хранения суммарных прогнозов
total_forecast_list = []

# Прогноз продаж для каждой номенклатуры
for item, model in models.items():
    forecast_sales = model.predict(forecast_data[['Цена']])
    total_forecast_sales = sum(forecast_sales)
    total_forecast_list.append((item, total_forecast_sales))

# Создание датафрейма с суммарными прогнозами
total_forecast_df = pd.DataFrame(total_forecast_list, columns=['Номенклатура', 'Суммарный прогноз'])

# Сохранение результатов прогноза в Excel
total_forecast_df.to_excel("прогноз_продаж.xlsx", index=False)

print("Прогноз продаж сохранен в файл 'прогноз_продаж.xlsx'")


#  XGBoost

In [ ]:
import pandas as pd                  # ver.2 заниженные прогнозы  XGBoost
from xgboost import XGBRegressor
from datetime import datetime, timedelta

# Загрузка данных о продажах из Excel
df = pd.read_excel("sales.xlsx")
df['Дата'] = pd.to_datetime(df['Дата'], format="%d.%m.%Y")

# Загрузка данных о ценах из Excel
prices_df = pd.read_excel("new_prices.xlsx")

# Получение последней даты в исходных данных
last_date = df['Дата'].max()

# Создание датафрейма с данными о ценах на все 14 дней
forecast_dates = [last_date + timedelta(days=i) for i in range(1, 15)]
forecast_data = pd.DataFrame({'Дата': forecast_dates})

# Дублируем данные о ценах для каждой номенклатуры
forecast_data['Цена'] = prices_df['Цена'].iloc[0]

# Разделение данных на признаки и целевую переменную
X = df[['Цена']]
y = df['Кол-во продано, шт.']

# Обучение модели градиентного бустинга для каждой номенклатуры
models = {}
for item in df['Номенклатура'].unique():
    item_df = df[df['Номенклатура'] == item]
    X_item = item_df[['Цена']]
    y_item = item_df['Кол-во продано, шт.']
    model = XGBRegressor(n_estimators=100, max_depth=10, random_state=42)
    model.fit(X_item, y_item)
    models[item] = model

# Создание списка для хранения суммарных прогнозов
total_forecast_list = []

# Прогноз продаж для каждой номенклатуры
for item, model in models.items():
    forecast_sales = model.predict(forecast_data[['Цена']])
    total_forecast_sales = sum(forecast_sales)
    total_forecast_list.append((item, total_forecast_sales))

# Создание датафрейма с суммарными прогнозами
total_forecast_df = pd.DataFrame(total_forecast_list, columns=['Номенклатура', 'Суммарный прогноз'])

# Сохранение результатов прогноза в Excel
total_forecast_df.to_excel("прогноз_продаж.xlsx", index=False)

print("Прогноз продаж сохранен в файл 'прогноз_продаж.xlsx'")


#  Lightgbm

In [ ]:
import pandas as pd
import lightgbm as lgb
from datetime import datetime, timedelta

# Загрузка данных о продажах из Excel
df = pd.read_excel("sales.xlsx")
df['Дата'] = pd.to_datetime(df['Дата'], format="%d.%m.%Y")

# Загрузка данных о ценах из Excel
prices_df = pd.read_excel("new_prices.xlsx")

# Получение последней даты в исходных данных
last_date = df['Дата'].max()

# Создание датафрейма с данными о ценах на все 14 дней
forecast_dates = [last_date + timedelta(days=i) for i in range(1, 15)]
forecast_data = pd.DataFrame({'Дата': forecast_dates})

# Дублируем данные о ценах для каждой номенклатуры
forecast_data['Цена'] = prices_df['Цена'].iloc[0]

# Разделение данных на признаки и целевую переменную
X = df[['Цена']]
y = df['Кол-во продано, шт.']

# Обучение модели LightGBM для каждой номенклатуры
models = {}
for item in df['Номенклатура'].unique():
    item_df = df[df['Номенклатура'] == item]
    X_item = item_df[['Цена']]
    y_item = item_df['Кол-во продано, шт.']
    lgb_train = lgb.Dataset(X_item, y_item)
    params = {'objective': 'regression',
              'metric': 'mse',
              'num_leaves': 30,
              'learning_rate': 0.01}
    model = lgb.train(params, lgb_train, num_boost_round=100)
    models[item] = model

# Создание списка для хранения суммарных прогнозов
total_forecast_list = []

# Прогноз продаж для каждой номенклатуры
for item, model in models.items():
    forecast_sales = model.predict(forecast_data[['Цена']])
    total_forecast_sales = sum(forecast_sales)
    total_forecast_list.append((item, total_forecast_sales))

# Создание датафрейма с суммарными прогнозами
total_forecast_df = pd.DataFrame(total_forecast_list, columns=['Номенклатура', 'Суммарный прогноз'])

# Сохранение результатов прогноза в Excel
total_forecast_df.to_excel("прогноз_промо_продаж.xlsx", index=False)

print("Прогноз продаж сохранен в файл 'прогноз_промо_продаж.xlsx'")


# SARIMA

In [ ]:
%%time
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Загрузка данных из Excel
sales_data = pd.read_excel('Продажи.xlsx', sheet_name='Продажи')
price_data = pd.read_excel('Продажи.xlsx', sheet_name='Цены')

# Подготовка данных
sales_data['Дата'] = pd.to_datetime(sales_data['Дата'], format="%d.%m.%Y")
sales_data.set_index('Дата', inplace=True)

# Создание списка уникальных товаров
unique_items = sales_data['Номенклатура'].unique()

# Прогнозирование и сохранение результатов для каждого товара
forecast_results = []

for item in unique_items:
    # Фильтрация данных для текущего товара
    item_sales_data = sales_data[sales_data['Номенклатура'] == item]

    if len(item_sales_data) < 2:
        # Пропускаем товары с недостаточным количеством данных для прогноза
        continue

    # Последняя известная цена для текущего товара
    last_price = price_data[price_data['Номенклатура'] == item]['Цена'].iloc[-1]

    # Повторение цены для каждого дня прогноза
    exog_forecast = np.full((14, 1), last_price)

    try:
        # Построение модели SARIMA для текущего товара с учетом последней известной цены
        model = SARIMAX(item_sales_data['Кол-во продано, шт.'], exog=item_sales_data['Цена'], order=(1, 0, 1), seasonal_order=(1, 0, 1, 7))
        result = model.fit()

        # Прогноз на 14 дней вперед, используя последнюю известную цену
        forecast = result.get_forecast(steps=14, exog=exog_forecast)
        total_forecast = forecast.predicted_mean.sum()

        # Добавление результатов прогноза для текущего товара в список
        forecast_results.append({'Код Товара': item, 'Прогноз': total_forecast})
    except:
        # Пропускаем товары, для которых не удалось построить модель SARIMA
        continue

# Создание DataFrame с результатами прогноза
forecast_df = pd.DataFrame(forecast_results)

# Сохранение результатов в Excel
forecast_df.to_excel('прогноз_продаж_по_товарамSARIMA.xlsx', index=False)

print("Прогноз продаж по каждому товару сохранен в файл 'прогноз_продаж_по_товарам.xlsx'")


# LSTM (RNN-рекурентные нейронные сети) ,аналог GRU